In [ ]:
from feedback_synthesizer.inference import Synthesizer
from IPython.display import Audio
import tensorflow as tf
import os
import numpy as np
from pathlib import Path
import librosa
import random
from tqdm import tqdm
from vocoder.inference import load_model, infer_waveform

os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
wavrnn = load_model('vocoder/saved_models/vctk/vctk.pt')

In [ ]:
Synthesizer.hparams.tacotron_num_gpus = 1
Synthesizer.hparams.postnet_num_layers = 5
Synthesizer.hparams.outputs_per_step=2

checkpoints_dir = 'feedback_synthesizer/saved_models/logs-vctk/taco_pretrained/'
checkpoints_dir = Path(checkpoints_dir)
synthesizer=Synthesizer(checkpoints_dir,low_mem=False)

In [ ]:
texts=['There were many editions of these works still being used in the nineteenth century.']

# validation set: p260_006 p260_004
# test set: p340_006 p231_010
name = 'p340_006' 

embed=np.load('datasets/vctk/synthesizer/embeds/embed-' + name + '.npy')
embeds = np.stack([embed] * len(texts))
specs = synthesizer.synthesize_spectrograms(texts, embeds)
breaks = [spec.shape[1] for spec in specs]

spec = np.concatenate(specs, axis=1)

assert spec is not None

wav = Synthesizer.griffin_lim(spec)

In [ ]:
b_ends = np.cumsum(np.array(breaks) * Synthesizer.hparams.hop_size)
b_starts = np.concatenate(([0], b_ends[:-1]))
wavs = [wav[start:end] for start, end, in zip(b_starts, b_ends)]
breaks = [np.zeros(int(0.15 * Synthesizer.sample_rate))] * len(breaks)
wav = np.concatenate([i for w, b in zip(wavs, breaks) for i in (w, b)])
wav = wav / np.abs(wav).max() * 0.97

In [ ]:
Audio(wav,rate=Synthesizer.sample_rate)

In [ ]:
neural_wav = infer_waveform(spec)
Audio(wav,rate=Synthesizer.sample_rate)

In [ ]:
ori_wav = np.load('datasets/vctk/synthesizer/audio/audio-' + name + '.npy')
Audio(ori_wav, rate=16000)